In [1]:
!pip install datasets 
!pip install transformers==4.18.0 
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from datasets import *
from transformers import *
from tokenizers import *
import os
import json
import pandas as pd

In [3]:
dataset = load_dataset("cc_news", split="train")

Generating train split:   0%|          | 0/708241 [00:00<?, ? examples/s]

Dataset cc_news downloaded and prepared to /root/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/ae469e556251e6e7e20a789f93803c7de19d0c4311b6854ab072fecb4e401bd6. Subsequent calls will reuse this data.


In [4]:
print(dataset)
print(type(dataset))

Dataset({
    features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url'],
    num_rows: 708241
})
<class 'datasets.arrow_dataset.Dataset'>


In [5]:
# split the dataset into training (90%) and testing (10%)
d = dataset.train_test_split(test_size=0.1)
print(d)
#d["train"], d["test"]

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url'],
        num_rows: 637416
    })
    test: Dataset({
        features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url'],
        num_rows: 70825
    })
})


In [6]:
df_ = pd.read_csv("/content/drive/MyDrive/fyp/fyp2/Bitcoin_tweets_cleaned_no_duplicates.csv")
display(df_)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,date,tweet,is_retweet
0,0,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after bi...,False
1,1,2021-02-10 23:58:48,"😎 Today, that's this Thursday, we will do a ""🎬...",False
2,2,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",False
3,4,2021-02-10 23:54:06,BTC in too big to fail…,False
4,6,2021-02-10 23:53:17,&lt;'fire' &amp; 'man'&gt;\nBitcoin Crypto BTC,False
...,...,...,...,...
2733687,3543846,2022-05-30 00:00:04,📄Daily BTC Liquidation Summary📄\n\n Liquidatio...,False
2733688,3543847,2022-05-30 00:00:04,⭕️Daily BTC Spot Summary⭕️ \n Total Trades\n🟢B...,False
2733689,3543848,2022-05-30 00:00:03,1790.98\n\n03 UTC\n03 JST\n &gt; 29430.0 la...,False
2733690,3543850,2022-05-30 00:00:02,ASTRO BOT ALE$BTCUSDT\n\n2h\n\n LONG(Buy) from...,False


In [23]:
valid_percentage = 0.2
import datasets
from datasets import Dataset


df = df_.copy()
df = df[0:10000]
df = df.dropna()
df = df.rename(columns={"tweet": "text"})
display(df)
df['label'] = 1
df = df[['text','label']]
#valid_size_each_class = int(df.shape[0] * valid_percentage)
#df = df.to_frame().T

print(type(df))
display(df)


valid_size_ = int(df.shape[0] * valid_percentage)




validation_dataset = df[0:valid_size_]
validation_dataset = validation_dataset.sample(frac=1).reset_index(drop=True)

training_dataset = df[valid_size_:]
training_dataset = training_dataset.sample(frac=1).reset_index(drop=True)



train_dataset = Dataset.from_dict(training_dataset)
validation_dataset = Dataset.from_dict(validation_dataset)
d = datasets.DatasetDict({"train":train_dataset,"test":validation_dataset})

print(type(d))
print(d)

,Unnamed: 0,date,text,is_retweet
0,0,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after bi...,False
1,1,2021-02-10 23:58:48,"😎 Today, that's this Thursday, we will do a ""🎬...",False
2,2,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",False
3,4,2021-02-10 23:54:06,BTC in too big to fail…,False
4,6,2021-02-10 23:53:17,&lt;'fire' &amp; 'man'&gt;\nBitcoin Crypto BTC,False
...,...,...,...,...
9995,12005,2021-02-08 13:00:47,\n----------------------------\n 2021-02-08 |...,False
9996,12006,2021-02-08 13:00:43,$TSLA &amp; Pappa Elon for the W BTC Bitcoin,False
9997,12007,2021-02-08 13:00:43,What a hike! Elon Musk's Tesla buys $1.5 billi...,False
9998,12008,2021-02-08 13:00:40,10 $BTC sold into the bid @ 43450\n 👈 Bitco...,False


<class 'pandas.core.frame.DataFrame'>


,text,label
0,Blue Ridge Bank shares halted by NYSE after bi...,1
1,"😎 Today, that's this Thursday, we will do a ""🎬...",1
2,"Guys evening, I have read this article about B...",1
3,BTC in too big to fail…,1
4,&lt;'fire' &amp; 'man'&gt;\nBitcoin Crypto BTC,1
...,...,...
9995,\n----------------------------\n 2021-02-08 |...,1
9996,$TSLA &amp; Pappa Elon for the W BTC Bitcoin,1
9997,What a hike! Elon Musk's Tesla buys $1.5 billi...,1
9998,10 $BTC sold into the bid @ 43450\n 👈 Bitco...,1


<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [24]:
for t in d["train"]["text"][:3]:
  print(t)
  print("="*50)

Update!!

Holo Holochain HOT cryptocurrency CryptoNews TechNews innovation investing Binance Bitcoin…
Bitwise files prospectus with SEC for an ETF to track 'crypto innovators'.  crypto BTC Bitcoin DOGE Binance
Coinbase is an Exchange that crushes commissions, it is unfortunate ... That is why I have decided to close my acco…


In [25]:
# if you want to train the tokenizer from scratch (especially if you have custom
# dataset loaded as datasets object), then run this cell to save it as files
# but if you already have your custom data as text files, there is no point using this
def dataset_to_text(dataset, output_filename="data.txt"):
  """Utility function to save dataset text to disk,
  useful for using the texts to train the tokenizer 
  (as the tokenizer accepts files)"""
  with open(output_filename, "w") as f:
    for t in dataset["text"]:
      print(t, file=f)

# save the training set to train.txt
dataset_to_text(d["train"], "train.txt")
# save the testing set to test.txt
dataset_to_text(d["test"], "test.txt")

In [26]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]
# if you want to train the tokenizer on both sets
# files = ["train.txt", "test.txt"]
# training the tokenizer on the training set
files = ["train.txt"]
# 30,522 vocab is BERT's default vocab size, feel free to tweak
vocab_size = 30_522
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 512
# whether to truncate
truncate_longer_samples = False

In [27]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from pathlib import Path


tokenizer = ByteLevelBPETokenizer(lowercase=True)

# Customize training
tokenizer.train(files=files, vocab_size=8192, min_frequency=2,
                show_progress=True,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
])
#Save the Tokenizer to disk
tokenizer.save_model("/content/")

['/content/vocab.json', '/content/merges.txt']

In [28]:
model_name = "svalabs/twitter-xlm-roberta-bitcoin-sentiment"

In [29]:
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM

# Set a configuration for our RoBERTa model
config = RobertaConfig(
    vocab_size=8192,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
# Initialize the model from a configuration without pretrained weights
model = RobertaForMaskedLM(config=config)
print('Num parameters: ',model.num_parameters())

Num parameters:  49816064


In [30]:
from transformers import RobertaTokenizerFast
# Create the tokenizer from a trained one
tokenizer = RobertaTokenizerFast.from_pretrained("/content/", max_len=512)

Didn't find file /content/tokenizer.json. We won't load it.
Didn't find file /content/added_tokens.json. We won't load it.
Didn't find file /content/special_tokens_map.json. We won't load it.
Didn't find file /content/tokenizer_config.json. We won't load it.
loading file /content/vocab.json
loading file /content/merges.txt
loading file None
loading file None
loading file None
loading file None


In [31]:
from transformers import DataCollatorForLanguageModeling

# Define the Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [32]:

from transformers import Trainer, TrainingArguments
TRAIN_EPOCHS = 5
#LEARNING_RATE = 
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
# Define the training arguments
training_args = TrainingArguments(
    output_dir='/content/',
    overwrite_output_dir=True,
    evaluation_strategy = 'epoch',
    num_train_epochs=TRAIN_EPOCHS,
    #learning_rate=LEARNING_RATE,
    #weight_decay=WEIGHT_DECAY,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    save_steps=8192,
    #eval_steps=4096,
    save_total_limit=1,
)
# Create the trainer for our model
print(d['train'])
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=d['train'],
    eval_dataset=d['test'],
    #prediction_loss_only=True,
)
# Train the model
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: text. If text are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed 

Dataset({
    features: ['text', 'label'],
    num_rows: 8000
})


ValueError: ignored